In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import matplotlib.pyplot as plt  
import re
from sklearn import model_selection

data_train = pd.read_csv("train.csv")
#data_train

In [2]:
def set_Cabin_type(df):
    df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
    df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"
    return df


In [3]:
# 补全Cabin`
set_Cabin_type(data_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,No,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,Yes,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,No,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,Yes,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,No,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,No,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,Yes,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,No,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,No,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,No,C


In [4]:
# what is each person's title? 
data_train['Title'] = data_train['Name'].map(lambda x: re.compile(", (.*?)\.").findall(x)[0])

# 将各式称呼进行统一化处理：
title_Dict = {}
title_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
title_Dict.update(dict.fromkeys(['Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty'))
title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
title_Dict.update(dict.fromkeys(['Mr','Jonkheer'], 'Mr'))
title_Dict.update(dict.fromkeys(['Master'], 'Master'))
data_train['Title'] = data_train['Title'].map(title_Dict)

# #使用dummy对不同的称呼进行分列：
dummies_Title = pd.get_dummies(data_train['Title'], prefix= 'Title')
data_train = pd.concat([data_train, dummies_Title], axis=1)
#data_train

In [5]:
# 补全Age
master_mean_age = data_train.loc[(data_train.Title_Master == 1), 'Age'].mean()
data_train.loc[(data_train.Age.isnull()) & (data_train.Title_Master == 1), 'Age' ] = master_mean_age

miss_mean_age = data_train.loc[(data_train.Title_Miss == 1), 'Age'].mean()
data_train.loc[(data_train.Age.isnull()) & (data_train.Title_Miss == 1), 'Age' ] = miss_mean_age

mr_mean_age = data_train.loc[(data_train.Title_Mr == 1), 'Age'].mean()
data_train.loc[(data_train.Age.isnull()) & (data_train.Title_Mr == 1), 'Age' ] = mr_mean_age

mrs_mean_age = data_train.loc[(data_train.Title_Mrs == 1), 'Age'].mean()
data_train.loc[(data_train.Age.isnull()) & (data_train.Title_Mrs == 1), 'Age' ] = mrs_mean_age

officer_mean_age = data_train.loc[(data_train.Title_Officer == 1), 'Age'].mean()
data_train.loc[(data_train.Age.isnull()) & (data_train.Title_Officer == 1), 'Age' ] = officer_mean_age

royalty_mean_age = data_train.loc[(data_train.Title_Royalty == 1), 'Age'].mean()
data_train.loc[(data_train.Age.isnull()) & (data_train.Title_Royalty == 1), 'Age' ] = royalty_mean_age

In [6]:
# 新建一个Child特征
data_train['Child'] = (data_train.Age < 12) + 0
#data_train.info()

In [7]:
import sklearn.preprocessing as preprocessing
# 完成Age与Fare的归一化
scaler = preprocessing.StandardScaler()

age_scale_param = scaler.fit(np.array(data_train.Age).reshape(-1,1))
data_train['Age_scaled'] = scaler.fit_transform(np.array(data_train.Age).reshape(-1,1), age_scale_param)

fare_scale_param = scaler.fit(np.array(data_train.Fare).reshape(-1,1))
data_train['Fare_scaled'] = scaler.fit_transform(np.array(data_train.Fare).reshape(-1,1), fare_scale_param)
#data_train.info()

In [8]:
dummies_Cabin = pd.get_dummies(data_train['Cabin'], prefix= 'Cabin')
dummies_Embarked = pd.get_dummies(data_train['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(data_train['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(data_train['Pclass'], prefix= 'Pclass')

df = pd.concat([data_train, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 25 columns):
PassengerId      891 non-null int64
Survived         891 non-null int64
Age              891 non-null float64
SibSp            891 non-null int64
Parch            891 non-null int64
Fare             891 non-null float64
Title_Master     891 non-null uint8
Title_Miss       891 non-null uint8
Title_Mr         891 non-null uint8
Title_Mrs        891 non-null uint8
Title_Officer    891 non-null uint8
Title_Royalty    891 non-null uint8
Child            891 non-null int64
Age_scaled       891 non-null float64
Fare_scaled      891 non-null float64
Cabin_No         891 non-null uint8
Cabin_Yes        891 non-null uint8
Embarked_C       891 non-null uint8
Embarked_Q       891 non-null uint8
Embarked_S       891 non-null uint8
Sex_female       891 non-null uint8
Sex_male         891 non-null uint8
Pclass_1         891 non-null uint8
Pclass_2         891 non-null uint8
Pclass_3         891 no

## 计算交叉验证集 

In [10]:
# 简单看看打分情况
from sklearn import model_selection
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
all_data = df.filter(regex='Survived|Age_.*|Sibsp|Parch|Fare_scaled|Carbin_*|Sex_*|Pclass_*|Child|Title_*')

X = np.array(all_data.values[:,1:])
y = np.array(all_data.values[:,0]).reshape(-1,1).astype(int)
X = pd.DataFrame(X)
y = pd.DataFrame(y)
y = y.iloc[:,0].map(int)
# k折交叉验证
# 验证某个模型在某个训练集上的稳定性，输出cv=k个预测精度
# 把初始训练样本分成k份，其中（k-1）份被用作训练集，剩下一份被用作评估集，这样一共可以对分类器做k次训练，并且得到k个训练结果。
print(model_selection.cross_val_score(clf, X, y, cv=5))

[0.7877095  0.81005587 0.79775281 0.80337079 0.83615819]


## 用全部训练集来训练一个模型 

In [11]:
from sklearn import linear_model

# 用正则取出我们要的属性值
train_df = df.filter(regex='Survived|Age_.*|Sibsp|Parch|Fare_scaled|Carbin_*|Sex_*|Pclass_*|Child|Title_*')
train_np = train_df.values

# y即Survival结果
y = train_np[:, 0]

# X即特征属性值
X = train_np[:, 1:]

# fit到RandomForestRegressor之中
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(X, y)

clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [12]:
predictions = clf.predict(X)
result = (predictions.reshape(-1,1) == y.reshape(-1,1))
sum(result)


array([725])

## 开始预测

对测试集做相同的处理

其实应该和训练集一起初始化

二表合一

In [15]:
data_test = pd.read_csv("./test.csv")
data_test.loc[ (data_test.Fare.isnull()), 'Fare' ] = 0
# 接着我们对test_data做和train_data中一致的特征变换
data_test = set_Cabin_type(data_test)

data_test['Title'] = data_test['Name'].map(lambda x: re.compile(", (.*?)\.").findall(x)[0])

# 将各式称呼进行统一化处理：
title_Dict = {}
title_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
title_Dict.update(dict.fromkeys(['Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty'))
title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
title_Dict.update(dict.fromkeys(['Mr','Jonkheer'], 'Mr'))
title_Dict.update(dict.fromkeys(['Master'], 'Master'))
data_test['Title'] = data_test['Title'].map(title_Dict)


# #使用dummy对不同的称呼进行分列：
dummies_Title = pd.get_dummies(data_test['Title'], prefix= 'Title')
data_test = pd.concat([data_test, dummies_Title], axis=1)

# 补全Age
master_mean_age = data_test.loc[(data_test.Title_Master == 1), 'Age'].mean()
data_test.loc[(data_test.Age.isnull()) & (data_test.Title_Master == 1), 'Age' ] = master_mean_age

miss_mean_age = data_test.loc[(data_test.Title_Miss == 1), 'Age'].mean()
data_test.loc[(data_test.Age.isnull()) & (data_test.Title_Miss == 1), 'Age' ] = miss_mean_age

mr_mean_age = data_test.loc[(data_test.Title_Mr == 1), 'Age'].mean()
data_test.loc[(data_test.Age.isnull()) & (data_test.Title_Mr == 1), 'Age' ] = mr_mean_age

mrs_mean_age = data_test.loc[(data_test.Title_Mrs == 1), 'Age'].mean()
data_test.loc[(data_test.Age.isnull()) & (data_test.Title_Mrs == 1), 'Age' ] = mrs_mean_age

officer_mean_age = data_test.loc[(data_test.Title_Officer == 1), 'Age'].mean()
data_test.loc[(data_test.Age.isnull()) & (data_test.Title_Officer == 1), 'Age' ] = officer_mean_age

royalty_mean_age = data_test.loc[(data_test.Title_Royalty == 1), 'Age'].mean()
data_test.loc[(data_test.Age.isnull()) & (data_test.Title_Royalty == 1), 'Age' ] = royalty_mean_age

data_test['Child'] = (data_test.Age < 12) + 0
#data_test


In [16]:
import sklearn.preprocessing as preprocessing
# 完成Age与Fare的归一化
scaler = preprocessing.StandardScaler()

age_scale_param = scaler.fit(np.array(data_test.Age).reshape(-1,1))
data_test['Age_scaled'] = scaler.fit_transform(np.array(data_test.Age).reshape(-1,1), age_scale_param)

fare_scale_param = scaler.fit(np.array(data_test.Fare).reshape(-1,1))
data_test['Fare_scaled'] = scaler.fit_transform(np.array(data_test.Fare).reshape(-1,1), fare_scale_param)


In [17]:
dummies_Cabin = pd.get_dummies(data_test['Cabin'], prefix= 'Cabin')
dummies_Embarked = pd.get_dummies(data_test['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(data_test['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(data_test['Pclass'], prefix= 'Pclass')

df_test = pd.concat([data_test, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
# #df_test.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Title'], axis=1, inplace=True)
# df_test['Age_scaled'] = scaler.fit_transform(np.array(df_test.Age).reshape(-1,1), age_scale_param)
# df_test['Fare_scaled'] = scaler.fit_transform(np.array(df_test.Fare).reshape(-1,1), fare_scale_param)
test = df_test.filter(regex='Age_.*|Sibsp|Parch|Fare_.*|Carbin_.*|Sex_.*|Pclass_.*|Child|Title_.*')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
Parch            418 non-null int64
Title_Master     418 non-null uint8
Title_Miss       418 non-null uint8
Title_Mr         418 non-null uint8
Title_Mrs        418 non-null uint8
Title_Officer    418 non-null uint8
Title_Royalty    418 non-null uint8
Child            418 non-null int64
Age_scaled       418 non-null float64
Fare_scaled      418 non-null float64
Sex_female       418 non-null uint8
Sex_male         418 non-null uint8
Pclass_1         418 non-null uint8
Pclass_2         418 non-null uint8
Pclass_3         418 non-null uint8
dtypes: float64(2), int64(2), uint8(11)
memory usage: 17.6 KB


开始预测

In [18]:
predictions = clf.predict(test)
result = pd.DataFrame({'PassengerId':data_test['PassengerId'].values, 'Survived':predictions.astype(np.int32)})
result.to_csv("./logistic_regression_predictions_v1.csv", index=False)

### Model ensembling

In [19]:
from sklearn.ensemble import BaggingRegressor

# fit到BaggingRegressor之中
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
bagging_clf = BaggingRegressor(clf, n_estimators=20, max_samples=0.8, max_features=1.0, bootstrap=True, bootstrap_features=False, n_jobs=-1)
bagging_clf.fit(X, y)

predictions = bagging_clf.predict(test)
result = pd.DataFrame({'PassengerId':data_test['PassengerId'].values, 'Survived':predictions.astype(np.int32)})
result.to_csv("./logistic_regression_bagging_predictions_v1.csv", index=False)